In [103]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [104]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
df.sample(10)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
530,25/11/2020 11:03:17,NaN,Male,Leipzig,Frontend Developer,NaN,6,Middle,JavaScript,PHP,40800.0,3400,28,Full-time employee,Unlimited contract,German,101-1000,Product
675,25/11/2020 18:58:56,35.0,Male,Berlin,Software Engineer,1,1,Junior,FBD,"PHP, Javascript / Typescript, SQL",30000.0,NaN,26,Full-time employee,Unlimited contract,German,up to 10,Product
1102,06/12/2020 21:17:41,32.0,Female,Berlin,Computational linguist,5,5,Middle,Java,"Python, Docker",50400.0,0,29,Full-time employee,Unlimited contract,English,11-50,Product
174,24/11/2020 12:28:08,46.0,Male,Munich,QA Engineer,13,13,Senior,Java,SQL,70000.0,8500,28,Full-time employee,Unlimited contract,German,101-1000,Product
244,24/11/2020 14:04:28,26.0,Male,Munich,ML Engineer,1.5,1.5,Junior,Python,"C/C++, PyTorch",58000.0,NaN,30,Full-time employee,Unlimited contract,German,101-1000,Consulting / Agency
601,25/11/2020 14:08:39,29.0,Female,Berlin,Data Scientist,0,0,intern,Python,"Python, SQL, AWS, Google Cloud, Docker",20000.0,20000,25,Full-time employee,Temporary contract,English,1000+,Product
472,25/11/2020 01:29:48,31.0,Female,Berlin,QA Engineer,11,1.5,Middle,JS,"Ruby, SQL",57000.0,bvg only,30,Full-time employee,Unlimited contract,English,101-1000,Startup
217,24/11/2020 13:25:44,27.0,Male,Berlin,Backend Developer,5,1,Middle,Java,"AWS, Docker",65000.0,NaN,30,Full-time employee,Unlimited contract,English,1000+,Product
778,26/11/2020 17:08:47,42.0,Male,Berlin,Frontend Developer,18,5,Senior,React JS,Javascript / Typescript,850000.0,95000,35,Full-time employee,Unlimited contract,English,101-1000,Product
971,30/11/2020 15:45:24,29.0,Male,Berlin,DevOps,11,7,Lead,TypeScript,"Python, Javascript / Typescript, SQL, Google C...",140000.0,0,5,Self-employed (freelancer),Unlimited contract,English,up to 10,Startup


In [106]:
#df.info()
df["Other technologies/programming languages you use often"].isna().value_counts()

False    1096
True      157
Name: Other technologies/programming languages you use often, dtype: int64

In [107]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


In [108]:
df["Yearly bonus + stocks in EUR"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"], downcast="float", errors="coerce")
df["Yearly bonus + stocks in EUR"].fillna(0, inplace=True)
df['Total Compensation'] = df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']
df.drop(columns={"Timestamp","Yearly brutto salary (without bonus and stocks) in EUR", "Yearly bonus + stocks in EUR"}, inplace=True)
df['Position '].rename('Position')

0                            Software Engineer
1                            Backend Developer
2                            Software Engineer
3                           Frontend Developer
4                            Backend Developer
                         ...                  
1248                         Backend Developer
1249    Researcher/ Consumer Insights Analyst 
1250                     IT Operations Manager
1251                        Frontend Developer
1252                                    DevOps
Name: Position, Length: 1253, dtype: object

In [109]:
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp

In [110]:
df["Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10, "Other")
df['Other technologies/programming languages you use often'] = replace_low_freq(df, "Your main technology / programming language", 5, "Other")

In [111]:
#Age, Gender,Position ,Total years of experience,Years of experience in Germany,Seniority level,Number of vacation days,Employment status,Contract duration,Main language at work,Comapny size,Company type
df['Age'].fillna(df['Age'].median(), inplace=True)

df['Gender'].fillna('Other',inplace=True)
df['Position '].fillna('Other',inplace=True)
df['Seniority level'].fillna('Other',inplace=True)
df['Company type'].fillna('Other',inplace=True)

df['Employment status'].fillna('Unknown',inplace=True)
df['Сontract duration'].fillna('Unknown',inplace=True)
df['Main language at work'].fillna('Unknown',inplace=True)
df['Company size'].fillna('Unknown',inplace=True)
df['Company type'].fillna('Unknown',inplace=True)

df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], downcast="float", errors="coerce")
df['Years of experience in Germany'].fillna(0,inplace=True)

df['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'], downcast="float", errors="coerce")
df['Number of vacation days'].fillna(0,inplace=True)

df['Total years of experience'] = pd.to_numeric(df['Total years of experience'], downcast="float", errors="coerce")
df['Total years of experience'].fillna(df['Years of experience in Germany'],inplace=True)

df["Your main technology / programming language"].fillna('Unknown',inplace=True)
df['Other technologies/programming languages you use often'].fillna('Unknown',inplace=True)

In [112]:
y = (df['Total Compensation'])
x = df.drop(columns={'Total Compensation'})

xTrain,xTest,yTrain,yTest = train_test_split(x,y)

categorical_features = ['Gender', 'City', 'Position ','Seniority level','Your main technology / programming language',
                        'Other technologies/programming languages you use often','Employment status','Сontract duration',
                        'Main language at work','Company size', 'Company type']
categorical_transformer = OneHotEncoder()

numeric_features = ["Age", 'Total years of experience','Years of experience in Germany','Number of vacation days']
numeric_transformer = Pipeline( steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", MinMaxScaler())
    ])

preprocessor = ColumnTransformer( transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

pipeline_steps = [('pre', preprocessor),('grid', SGDRegressor())]
grid = Pipeline(pipeline_steps)

grid.fit(xTrain,yTrain)
grid.score(xTest,yTest)

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1527: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


ValueError: Found unknown categories ['Warsaw, Poland', 'Saarbrücken', 'NJ, USA', 'Cupertino', 'Hildesheim', 'Bodensee', 'Würzburg', 'Bölingen', 'Friedrichshafen', 'Milano', 'Prefer not to say', 'Dresden', 'Innsbruck', 'Tampere (Finland)', 'Sofia', 'Dortmund', 'Konstanz area', 'Heilbronn', 'Nuremberg ', 'Bucharest', 'Paderborn', 'Brunswick'] in column 1 during transform

In [39]:
model = LinearRegression(n_jobs=-1)
imputer_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean"))
])

numeric_transformer = Pipeline(steps=[
        ("scaler", MinMaxScaler())
])

preprocessor = Pipeline(steps=[
        ('imp', imputer_transformer),
        ("num", numeric_transformer),
])

pipeline_steps = [('pre', preprocessor),('LR', model)]

best = Pipeline(pipeline_steps)
#best.fit(xTrain,yTrain)

# Answers and Explainations
(Expand/modify as needed)

### Results

### Feature Selection Activities

### Hyperparameter Changes